# Molecular QTL workflow

In [ ]:
%revisions -s -n 10

## Data

Molecular QTL data from [Yang et al (2016) Science](http://eqtl.uchicago.edu/jointLCL/). Input are genotypes of ~100 YRI samples with their molecular QTL data measured in LCL.

- alternative splicing (AS) data is of the primary interest here.

### Genotypes

[Genotype data for YRI](http://eqtl.uchicago.edu/jointLCL/genotypesYRI.gen.txt.gz) is the conventional VCF format but has dosage for genotypes.

### Phenotypes

Phenotype data has format:

```
#Chr	start	end	ID	18486	18487	18488	18489	18498	18499
chr1	880180	880422	chr1:880180:880422:clu_15502	0.201694364955	0.665990212763	-1.21881815589	-0.342480185427	0.165404160483	-1.58524292941
```

The first 4 columns are genomic coordinates info. Others are molecular QTL in samples.

We analyze:

1. [Alternative splicing](http://eqtl.uchicago.edu/jointLCL/qqnorm_ASintron_RNAseqGeuvadis.txt)

## Analysis plan

- For each analysis unit (gene, or intron cluster for AS), get the 1MB up/down-stream variants in genotypes
- Remove top phenotype PC from phenotype data
- Fine-mapping using various methods. SuSiE for starters

## Workflow overview

In [ ]:
!sos run 20180704_MolecularQTL_Workflow.ipynb -h

In [ ]:
[global]
# Specify work directory
parameter: cwd = path("~/GIT/LargeFiles/AS_output")
# X data, the genotype VCF file path
parameter: x_data = path()
# Y data, the phenotype file paths
parameter: y_data = path()
# HDF5 file, merged X and Y data
parameter: h5_data = path()
fail_if(not x_data.is_file() and not h5_data.is_file(), msg = 'Please provide ``--x-data`` or ``--h5-data``!')
fail_if(not y_data.is_file() and not h5_data.is_file(), msg = 'Please provide ``--y-data`` or ``--h5-data``!')
pop = 'YRI'

## Regressing out top PCs on phenotype

There may well be better approach to control for covariates etc, but [here](https://github.com/bmvdgeijn/WASP/blob/master/examples/example_data/H3K27ac/get_PCs.R) is workflow from the authors and was deemed sufficient. See their supplemental table of Yang et al 2016 for how many PC to use for each molecular QTLs.

Need to cope with missing phenotype data here. See `na.omit` function call in `prcomp` and `na.actions=na.exclude`.

In [ ]:
# PCA on phenotype and remove top PCs
[preprocess_1 (Remove top phenotype PC)]
# Num. PC to remove from phenotype
parameter: num_pcs = 3 # Table S2 of NIHMS835311-supplement-supplement.pdf
# column name patter for `grep` in R to select phenotype columns
# eg. "^NA[0-9]+" to extract sample names
parameter: colname_pattern = '^[0-9]+'
input: y_data
output: f"{cwd}/{_input:bn}.PC{num_pcs}.removed.gz"
R: expand = "${ }", workdir = cwd, stdout = f"{_output:n}.stdout"
    num_pcs = ${num_pcs}
    dat <- read.table(${_input:r}, header=T, comment.char='', check.names=F)
    phenotype.matrix <- dat[,5:ncol(dat)]
    # extract columns of interest
    phenotype.matrix <- phenotype.matrix[,grep("${colname_pattern}", colnames(phenotype.matrix), value = T)]
    # perform principal component analysis
    pca <- prcomp(na.omit(phenotype.matrix))
    # PCA summary
    print(summary(pca))
    cat("output", num_pcs, "PCs \n")
    # remove top PC from phenotype; takes a while
    cov_pcs <- pca$rotation[, 1:num_pcs]
    new.phenotype.matrix <- do.call(rbind, lapply(1:nrow(phenotype.matrix), function(i) residuals(lm(t(phenotype.matrix[i,]) ~ as.matrix(cov_pcs), na.action=na.exclude))))
    colnames(new.phenotype.matrix) <- colnames(phenotype.matrix)
    new.dat <- cbind(dat[,1:4], new.phenotype.matrix)
    colnames(new.dat)[1] <- 'chr'
    write.table(new.dat, gzfile(${_output:r}), sep="\t", quote=F, col.names=T, row.names=F)

## Extract per unit variables

In [ ]:
# this step provides VCF file index
[index_vcf: provides = '{filename}.gz.tbi']
depends: executable('tabix')
input: f"{filename}.gz"
bash: expand=True
   tabix -p vcf {_input}

# Extract cis-SNPs and make fine-mapping datasets
[preprocess_2 (Get per-unit dataset)]
depends: Py_Module('pysam'), Py_Module('pandas'), Py_Module('feather'), Py_Module('rpy2'), f"{x_data}.tbi"
# Maximum distance to site of interest, eg. 1Mb up/downstream to TSS for gene level QTL
parameter: max_dist = 1000000
chroms = [f'chr{x+1}' for x in range(22)]
input: for_each = 'chroms', concurrent = True
output: dynamic(glob.glob('{cwd}/{y_data:bnn}/chr*/*.rds'))
python: workdir = cwd, expand = "${ }"
    def read_header(gzfile):
        import gzip
        with gzip.open(gzfile,'r') as f:
            for line in f:
                res = [x.decode() for x in line.split()]
                break
        return res

    chrom = "${_chroms}"
    phenotype_id = [f'${pop}_{x}' for x in read_header(${_input:r})[4:]]
    vcf_id = [f'${pop}_{x}' for x in read_header(${x_data:r})[9:]]
    from pathlib import Path
    import pysam
    tbx = pysam.TabixFile(${x_data:r})    
    import pandas as pd, numpy as np
    from feather import write_dataframe
    qts = pd.read_csv(${_input:r}, sep = '\t')
    qts = qts.loc[qts['chr'] == chrom]
    #
    import os, time, tempfile
    start_time = time.time()
    i = 0
    cmds = ["library(feather)"]
    tf = tempfile.NamedTemporaryFile()
    for site in sorted(set(qts['start'].tolist())):
        if(i % 100 == 0):
            print('[chrom %s percent completed] %.1f (%.1f sec elapsed)' % (chrom, (float(i+1)/qts.shape[0])*100, time.time() - start_time))
            if len(cmds) > 1:
                with open(tf.name, 'w') as f:
                    f.write('\n'.join(cmds))
                os.system(f"Rscript {tf.name}")
                cmds = ["library(feather)"]            
        unit = qts.loc[qts['start'] == site]
        i += unit.shape[0]
        start = max(site - ${max_dist}, 0)
        end = site + ${max_dist}
        genotypes = np.array([row for row in tbx.fetch(chrom, start, end, parser=pysam.asTuple())])
        if len(genotypes) == 0:
            continue
        Y_data = unit.drop(["chr", "start", "end", "ID"], axis=1).T
        Y_data.columns = [x.replace(':', '_') for x in unit['ID']]
        Y_data.index = phenotype_id
        X_data = pd.DataFrame(genotypes[:,9:].T,
                              columns = ['_'.join(x) for x in genotypes[:,[2,0,1,3,4]]], 
                              index = vcf_id)
        merged = Y_data.join(X_data, how='inner').astype(np.float32)
        Path(f'${cwd}/${y_data:bnn}/{chrom}').mkdir(exist_ok=True, parents=True)
        basename = f'${cwd}/${y_data:bnn}/{chrom}/{chrom}_{site}_{max(unit["end"].tolist())}'
        write_dataframe(merged, basename + '.feather')
        cmds.append("saveRDS(read_feather('{0}'), '{1}');system('rm -f {0}')".format(basename + '.feather', basename + '.rds'))

## Finemapping with SuSiE

In [ ]:
# Run finemapping with SuSiE
[SuSiE (SuSiE analysis)]
depends: R_library('susieR')
parameter: maxL = 5
parameter: prior_var = 0.1
input: glob.glob(f'{cwd}/{y_data:bnn}/chr*/*.rds'), group_by = 1, concurrent = True
output: dynamic(glob.glob(f'{cwd}/{y_data:bnn}/SuSiE_CS_*/*.rds'))
R: expand = "${ }"
    library(susieR)
    dat = readRDS(${_input:r})
    n_y = length(grep("^chr", colnames(dat), value = T))
    Y = as.matrix(dat[,1:n_y,drop=F])
    X = as.matrix(dat[,(n_y+1):ncol(dat),drop=F])
    storage.mode(X) = 'double'
    storage.mode(Y) = 'double'
    bad = which(sapply(1:ncol(X), function(i) all(is.na(X[,i]))))
    snps = colnames(X)[-bad]
    X = X[,-bad]
    for (r in ncol(Y)) {
      keep_rows = which(!is.na(Y[,r]))
      x = X[keep_rows,]
      y = Y[,r][keep_rows]
      z_score = susieR:::calc_z(x,y)
      names(z_score) = snps
      fitted = susie(x, y,
               L=${maxL},
               estimate_residual_variance = TRUE, 
               prior_variance = ${prior_var}, 
               intercept=FALSE,
               tol=1e-3)
      sets = susie_get_CS(fitted,
                    coverage = 0.95,
                    X = x, 
                    min_abs_corr = 0.4)
      pip = susie_get_PIP(fitted, sets$cs_index)
      dirname = paste0('${cwd}/${y_data:bnn}/SuSiE_CS_', length(sets$cs_index), '/')
      system(paste("mkdir -p", dirname))
      saveRDS(list(z_score=z_score,fitted=fitted,sets=sets,pip=pip), paste0(dirname, colnames(Y)[r], '.rds'))
    }